# SINGLE SITE ANOMALY DETECTION AND CORRECTION
This script performs anomaly detection and correction for multiple sensors at a single monitoring site.
The script imports data, performs initial anomaly detection based on rules, uses models (ARIMA and 4 flavors of LSTM)
and associated thresholds to detect anomalies, aggregates for overall anomaly detection, and performs correction
based on ARIMA.

Site: Logan River at Main Street

Sensors: temperature, specific conductance, pH, dissolved oxygen

Created: Amber Jones, March 2021

## Import Libraries and Functions

In [1]:
import pandas as pd
from PyHydroQC import anomaly_utilities
from PyHydroQC import model_workflow
from PyHydroQC import rules_detect
from PyHydroQC import ARIMA_correct
from PyHydroQC import modeling_utilities
from PyHydroQC.model_workflow import ModelType

## Retrieve data
Creates an object with a data frame specific to each sensor.

Functions called: 
- get_data

In [2]:
site = 'MainStreet'
sensors = ['temp', 'cond', 'ph', 'do']
sensor_array = anomaly_utilities.get_data(sensors=sensors, filename='MS2018.csv', path='../LRO_data/')
for snsr in sensor_array:
    print(snsr + str(sensor_array[snsr]))
    

temp                      raw
datetime                 
2018-01-01 00:00:00  2.19
2018-01-01 00:15:00  2.15
2018-01-01 00:30:00  2.11
2018-01-01 00:45:00  2.07
2018-01-01 01:00:00  2.02
...                   ...
2018-12-31 22:45:00  0.91
2018-12-31 23:00:00  0.89
2018-12-31 23:15:00  0.86
2018-12-31 23:30:00  0.82
2018-12-31 23:45:00  0.79

[35029 rows x 1 columns]
cond                       raw
datetime                  
2018-01-01 00:00:00  381.9
2018-01-01 00:15:00  382.0
2018-01-01 00:30:00  382.2
2018-01-01 00:45:00  382.6
2018-01-01 01:00:00  382.7
...                    ...
2018-12-31 22:45:00  371.9
2018-12-31 23:00:00  372.1
2018-12-31 23:15:00  372.1
2018-12-31 23:30:00  372.1
2018-12-31 23:45:00  372.2

[35029 rows x 1 columns]
ph                      raw
datetime                 
2018-01-01 00:00:00  8.47
2018-01-01 00:15:00  8.47
2018-01-01 00:30:00  8.46
2018-01-01 00:45:00  8.46
2018-01-01 01:00:00  8.46
...                   ...
2018-12-31 22:45:00  8.48
2018-12-31 23:0

## Import Parameters

In [3]:
# Parameters may be specified in a parameters file or in this script
from PyHydroQC.parameters import site_params, LSTM_params, calib_params
for snsr in sensors:
    print(snsr + str(site_params[site][snsr]))
print('LSTM' + str(LSTM_params))
print('calib' + str(calib_params))


temp{'max_range': 20, 'min_range': -2, 'persist': 30, 'window_sz': 30, 'alpha': 1e-05, 'threshold_min': 0.4, 'widen': 1, 'pdq': [0, 0, 0]}
cond{'max_range': 2700, 'min_range': 150, 'persist': 30, 'window_sz': 40, 'alpha': 1e-06, 'threshold_min': 5.0, 'widen': 1, 'pdq': [1, 1, 5]}
ph{'max_range': 9.5, 'min_range': 7.5, 'persist': 45, 'window_sz': 20, 'alpha': 0.0001, 'threshold_min': 0.03, 'widen': 1, 'pdq': [3, 1, 1]}
do{'max_range': 15, 'min_range': 5, 'persist': 45, 'window_sz': 30, 'alpha': 1e-05, 'threshold_min': 0.25, 'widen': 1, 'pdq': [1, 1, 1]}
LSTM{'time_steps': 5, 'samples': 20000, 'cells': 128, 'dropout': 0.2, 'patience': 6}
calib{'persist_low': 3, 'persist_high': 7, 'hour_low': 7, 'hour_high': 17}


## Rules Based Anomaly Detection
Performs checks for range and persistence. Min/max range and duration are defined in the parameters.
Data outside a range or persisting longer than a duration are detected as anomalous, corrected by linear interpolation.
The output is a column 'observed' of intermediate results that are used for subsequent modeling.

Functions called: 
- range_check
- persistence
- interpolate

In [4]:
range_count = dict()
persist_count = dict()
rules_metrics = dict()
for snsr in sensor_array:
    sensor_array[snsr], range_count[snsr] = rules_detect.range_check(
        df=sensor_array[snsr], maximum=site_params[site][snsr]['max_range'], minimum=site_params[site][snsr]['min_range'])
    sensor_array[snsr], persist_count[snsr] = rules_detect.persistence(
        df=sensor_array[snsr], length=site_params[site][snsr]['persist'], output_grp=True)
    sensor_array[snsr] = rules_detect.interpolate(df=sensor_array[snsr])
print('Rules based detection complete.\n')

Rules based detection complete.



### Detect Calibration Events
Calibration events are identified where persistence is within a certain window (e.g., after a sensor is returned to
the water, it is 'stuck' and reports the same values for several time steps), the time of day within a certain window,
and where these overlap for all sensors. When this occurs, an event is identified. Hours and durations are defined in
the parameters. A subset of sensors are selected (1:4) because temperature is not calibrated. Calibration events 
detected here should be reviewed, compared to field records, and organized as input to the following step: 
drift correction.

Functions called: 
- calib_overlap
    - calib_detect

In [5]:
calib_sensors = sensors[1:4]
input_array = dict()
for snsr in calib_sensors:
    input_array[snsr] = sensor_array[snsr]
all_calib, all_calib_dates, df_all_calib, calib_dates_overlap = rules_detect.calib_overlap(
    sensor_names=calib_sensors, input_array=input_array, calib_params=calib_params)
print(str(calib_dates_overlap))

DatetimeIndex(['2018-01-11 14:45:00', '2018-01-11 15:00:00',
               '2018-01-18 07:15:00', '2018-01-18 07:30:00',
               '2018-01-18 07:45:00', '2018-01-19 07:45:00',
               '2018-01-19 08:00:00', '2018-01-19 08:15:00',
               '2018-01-19 08:30:00', '2018-01-30 07:30:00',
               '2018-01-30 07:45:00', '2018-02-02 06:45:00',
               '2018-02-02 07:00:00', '2018-02-02 07:15:00',
               '2018-04-17 12:45:00', '2018-05-03 15:00:00',
               '2018-05-03 15:15:00', '2018-05-03 15:30:00',
               '2018-05-17 14:30:00', '2018-05-17 14:45:00',
               '2018-05-17 15:00:00', '2018-05-17 15:15:00',
               '2018-05-17 15:30:00', '2018-05-17 15:45:00',
               '2018-07-04 11:45:00', '2018-07-04 12:00:00',
               '2018-07-04 12:15:00', '2018-10-08 14:30:00',
               '2018-10-08 14:45:00', '2018-10-08 15:00:00',
               '2018-10-08 15:15:00', '2018-10-08 15:30:00',
               '2018-10-

### Perform Linear Drift Correction
Drift correction is a correction used when data are shifted due to calibration or cleaning of a sensor. To perform drift
correction, the routine requires a start date, an end date, and a gap value to shift the data for each event. This step 
requires some manual effort either after calibration events are detected as above or by reviewing field records and raw 
data. In this case, the inputs were determined from the scripts that technicians previously ran on these data and are 
stored in a spreadsheet with columns for start date, end date, and gap value.

Functions called: 
- lin_drift_cor

In [6]:
calib_sensors = sensors[1:4]
calib_dates = dict()
for cal_snsr in calib_sensors:
    calib_dates[cal_snsr] = pd.read_csv(
        '../LRO_data/' + site + '_' + cal_snsr + '_calib_dates.csv', header=1, parse_dates=True, infer_datetime_format=True)
    calib_dates[cal_snsr]['start'] = pd.to_datetime(calib_dates[cal_snsr]['start'])
    calib_dates[cal_snsr]['end'] = pd.to_datetime(calib_dates[cal_snsr]['end'])
    calib_dates[cal_snsr] = calib_dates[cal_snsr].loc[(calib_dates[cal_snsr]['start'] > min(sensor_array[cal_snsr].index)) &
                                                      (calib_dates[cal_snsr]['start'] < max(sensor_array[cal_snsr].index))]
    if len(calib_dates[cal_snsr]) > 0:
        for i in range(min(calib_dates[cal_snsr].index), max(calib_dates[cal_snsr].index)):
            result, sensor_array[cal_snsr]['observed'] = rules_detect.lin_drift_cor(
                                                            observed=sensor_array[cal_snsr]['observed'],
                                                            start=calib_dates[cal_snsr]['start'][i],
                                                            end=calib_dates[cal_snsr]['end'][i],
                                                            gap=calib_dates[cal_snsr]['gap'][i],
                                                            replace=True)
print('Linear drift correction complete.')

Linear drift correction complete.


## Model Based Anomaly Detection
Generates 5 models. Each model predicts one step ahead using immediately adjacent data. Anomalies are detected by 
comparing the model residual (predictions - observations) to a threshold. Dynamic thresholds are based on model 
variability. Settings for ARIMA, LSTM, and threshold determination are in the parameters. Results from all models are 
aggregated so that a detection by any model results in an anomaly.

### ARIMA Detection
ARIMA models use a combination of past data in a linear form to predict the next value. These models are univariate.
Each ARIMA model requires the parameters p, d, q, which can be defined automatically as shown here.

Functions called: 
- pdq 
- ARIMA_detect: 
    - build_ARIMA_model
    - set_dynamic_threshold
    - detect_anomalies
    - anomaly_events

In [7]:
all_pdq = dict()
for snsr in sensors:
    all_pdq[snsr] = modeling_utilities.pdq(data=sensor_array[snsr]['observed'])
    print(snsr + ' (p, d, q) = ' + str(all_pdq[snsr]))
    site_params[site][snsr]['pdq'] = all_pdq[snsr]

ARIMA = dict()
for snsr in sensors:
    ARIMA[snsr] = model_workflow.ARIMA_detect(df=sensor_array[snsr], sensor=snsr, params=site_params[site][snsr],
                                              rules=False, plots=False, summary=False, compare=False)
print('\nARIMA detection complete.\n')

temp (p, d, q) = [3, 1, 1]
cond (p, d, q) = [5, 1, 4]
ph (p, d, q) = [5, 1, 3]
do (p, d, q) = [4, 1, 4]

Processing ARIMA detections.
temp ARIMA model complete.
Threshold determination complete.
ratio of detections: 0.034257 %

Processing ARIMA detections.
cond ARIMA model complete.
Threshold determination complete.
ratio of detections: 0.151303 %

Processing ARIMA detections.
ph ARIMA model complete.
Threshold determination complete.
ratio of detections: 0.054241 %

Processing ARIMA detections.
do ARIMA model complete.
Threshold determination complete.
ratio of detections: 0.085643 %

ARIMA detection complete.



### LSTM Detection
LSTM models create a neural network that uses a sequence of values to make a prediction. Settings and hyperparameters 
are defined in the parameters.


#### DATA: univariate, MODEL: vanilla
Uses a single sensor and data prior to predict the next point.

Functions called: 
- LSTM_detect_univar:
    - LSTM_univar:
        - create_scaler
        - create_sequenced_dataset
        - create_training_dataset
        - create_vanilla_model
        - train_model
    - set_dynamic_threshold
    - detect_anomalies
    - anomaly_events

In [8]:
LSTM_univar = dict()
for snsr in sensors:
    LSTM_univar[snsr] = model_workflow.LSTM_detect_univar(
            df=sensor_array[snsr], sensor=snsr, params=site_params[site][snsr], LSTM_params=LSTM_params, model_type=ModelType.VANILLA,
            rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)
print('\nLSTM univariate detection complete.\n')


Processing LSTM univariate ModelType.VANILLA detections.
1095/1095 [==============================] - 2s 2ms/step - loss: 0.7659

#### DATA: univariate,  MODEL: bidirectional
Uses a single sensor and data before and after to predict a point.

Functions called: 
- LSTM_detect_univar:
    - LSTM_univar_bidir:
        - create_scaler
        - create_bidir_sequenced_dataset
        - create_bidir_training_dataset
        - create_bidir_model
        - train_model
    - set_dynamic_threshold
    - detect_anomalies
    - anomaly_events

In [9]:
LSTM_univar_bidir = dict()
for snsr in sensors:
    LSTM_univar_bidir[snsr] = model_workflow.LSTM_detect_univar(
            df=sensor_array[snsr], sensor=snsr, params=site_params[site][snsr], LSTM_params=LSTM_params, model_type=ModelType.BIDIRECTIONAL,
            rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)
print('\nLSTM univariate bidirectional detection complete.\n')


Processing LSTM univariate ModelType.BIDIRECTIONAL detections.
1095/1095 [==============================] - 5s 4ms/step - loss: 0.7628

#### DATA: multivariate,  MODEL: vanilla
Uses multiple sensors as inputs and outputs and data prior to predict the next point.

Functions called: 
- LSTM_detect_multivar:
    - LSTM_multivar:
        - create_scaler
        - create_sequenced_dataset
        - create_training_dataset
        - create_vanilla_model
        - train_model
    - set_dynamic_threshold
    - detect_anomalies
    - anomaly_events

In [10]:
LSTM_multivar = model_workflow.LSTM_detect_multivar(
        sensor_array=sensor_array, sensors=sensors, params=site_params[site], LSTM_params=LSTM_params, model_type=ModelType.VANILLA,
        rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)
print('\nLSTM mutivariate detection complete.\n')


Processing LSTM multivariate ModelType.VANILLA detections.
Raw data shape: (35029, 4)
Observed data shape: (35029, 4)
Initial anomalies data shape: (35029, 4)
1095/1095 [==============================] - 3s 2ms/step - loss: 0.2916

#### DATA: multivariate,  MODEL: bidirectional
Uses multiple sensors as inputs and outputs and data before and after to predict a point.

Functions called: 
- LSTM_detect_multivar:
    - LSTM_multivar_bidir:
        - create_scaler
        - create_bidir_sequenced_dataset
        - create_bidir_training_dataset
        - create_bidir_model
        - train_model
    - set_dynamic_threshold
    - detect_anomalies
    - anomaly_events

In [12]:
LSTM_multivar_bidir = model_workflow.LSTM_detect_multivar(
        sensor_array=sensor_array, sensors=sensors, params=site_params[site], LSTM_params=LSTM_params, model_type=ModelType.BIDIRECTIONAL,
        rules=False, plots=False, summary=False, compare=False, model_output=False, model_save=False)
print('\nLSTM multivariate bidirectional detection complete.\n')

/Users/amber/.virtualenvs/LRO-anomaly-detection/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Processing LSTM multivariate ModelType.BIDIRECTIONAL detections.
Raw data shape: (35029, 4)
Observed data shape: (35029, 4)
Initial anomalies data shape: (35029, 4)
1095/1095 [==============================] - 4s 3ms/step - loss: 0.2659

### Aggregate Detections for All Models
Aggregates the results from all models.

Functions called: 
- aggregate_results

In [13]:
results_all = dict()
for snsr in sensors:
    models = dict()
    models['ARIMA'] = ARIMA[snsr].df
    models['LSTM_univar'] = LSTM_univar[snsr].df_anomalies
    models['LSTM_univar_bidir'] = LSTM_univar_bidir[snsr].df_anomalies
    models['LSTM_multivar'] = LSTM_multivar.all_data[snsr]
    models['LSTM_multivar_bidir'] = LSTM_multivar_bidir.all_data[snsr]
    results_all[snsr] = anomaly_utilities.aggregate_results(
        df=sensor_array[snsr], models=models, verbose=True, compare=False)

/Users/amber/.virtualenvs/LRO-anomaly-detection/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Correction
Correction is performed using piecewise ARIMA- small models determined for each period of anomalous data, along with
forecast and backcast, which consider data before and after a period of anomalous data and are merged together to 
ensure there are no discontinuities.

Functions called: 
- generate_corrections
    - group_bools
    - ARIMA_group
    - xfade

In [14]:
corrections = dict()
corrections = dict()
for snsr in sensors:
    corrections[snsr] = ARIMA_correct.generate_corrections(
        df=results_all[snsr], observed='observed', anomalies='detected_event', savecasts=True)
    print(snsr + ' correction complete.')

/Users/amber/.virtualenvs/LRO-anomaly-detection/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#########################################
